In [ ]:
import torch
import torch.nn as nn

class CNN2D(nn.Module):
    def __init__(self, num_classes):
        super(CNN2D, self).__init__()
        
        in_channels = 2
        # For the 2D conversion, we're assuming height=1 for the input
        height = 1
        width = 128  # Original length dimension
        
        # Upsample (adjusted for 2D)
        self.upsample = nn.Upsample(scale_factor=(1, 2), mode='bilinear', align_corners=False)  # height stays the same, width is doubled
        
        # Define 2D convolutional layers
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=(1, 3), padding=(0, 1))
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(1, 2))
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(1, 3), padding=(0, 1))
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(1, 2))
        
        # Adaptive pooling (2D)
        self.adaptive_avg_pool2d = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        
        # Define the fully connected layers (adjusted for adaptive pooling output)
        self.fc1 = nn.Linear(128, 256)  # The input features after pooling
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        # Adjust input shape for 2D processing
        x = x.unsqueeze(-2)  # Insert a height dimension of size 1
        
        # Upsample
        x = self.upsample(x)
        
        # 2D convolutional layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        # Adaptive average pooling
        x = self.adaptive_avg_pool2d(x)
        
        # Flatten (before fully connected layers)
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x
